In [41]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files='data/cleaned_data.csv')

# split into train, validation and test
train_test_dataset = dataset['train'].train_test_split(test_size=0.4)
val_test_dataset = train_test_dataset['test'].train_test_split(test_size=0.5)

dataset['train'] = train_test_dataset['train']
dataset['validation'] = val_test_dataset['train']
dataset['test'] = val_test_dataset['test']

In [42]:
dataset

DatasetDict({
    train: Dataset({
        features: ['course', 'book_id', 'topic', 'bookclub', 'chat_crew', 'pseudonym', 'message', 'time', 'is_answer', 'page', 'response_number', 'discussion_type', 'dialogic_spell', 'uptake', 'question', 'pivot', 'chat', 'chat_history'],
        num_rows: 543
    })
    validation: Dataset({
        features: ['course', 'book_id', 'topic', 'bookclub', 'chat_crew', 'pseudonym', 'message', 'time', 'is_answer', 'page', 'response_number', 'discussion_type', 'dialogic_spell', 'uptake', 'question', 'pivot', 'chat', 'chat_history'],
        num_rows: 181
    })
    test: Dataset({
        features: ['course', 'book_id', 'topic', 'bookclub', 'chat_crew', 'pseudonym', 'message', 'time', 'is_answer', 'page', 'response_number', 'discussion_type', 'dialogic_spell', 'uptake', 'question', 'pivot', 'chat', 'chat_history'],
        num_rows: 181
    })
})

In [96]:
messages = []
messages.append(("system", "Here is the chat history of the children discussion:"))
messages.append(("placeholder", "{history}"))
messages.append(("system", "Classify this sentence into one class of the codebook."))
messages.append(("human", "{input}"))

In [97]:
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_messages(messages)
prompt_template.format(input = {'input': 'ciao', 'chat_history': 'ciao'})

"System: Here is the chat history of the children discussion:\nSystem: Classify the sentence into one class of the codebook.\nHuman: {'input': 'ciao', 'chat_history': 'ciao'}"

In [140]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

dataset_csv_file = 'data/cleaned_data.csv'
text_field = 'message'
CLASS = 'discussion_type'
window_size = 5
combine_fields = ['pseudonym', 'message']
separator = ': '

data = pd.read_csv(dataset_csv_file)

data[text_field] = data[combine_fields].apply(lambda x: separator.join(x.dropna().astype(str)), axis=1)

history = []
for i in range(len(data)):
    if i >= 1 and not data.iloc[i][['book_id', 'bookclub', 'course']].equals(data.iloc[i-1][['book_id', 'bookclub', 'course']]):
        history = []

    data.at[i, 'history'] = '\n'.join(history) if history else pd.NA

    history.append(data.iloc[i][text_field])
    if len(history) > window_size:
        history.pop(0)

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

requests = train_data.apply(lambda x: {'input': x[text_field], 'history': [("human", chat) for chat in x['history'].split('\n')] if not pd.isna(x['history']) else []}, axis = 1)
prompt_template = ChatPromptTemplate.from_messages(messages)
prompts = prompt_template.batch(list(requests))
prompts = list(map(lambda x: x.to_string(), prompts))

dataset_dict = DatasetDict({
    'train': Dataset.from_dict({'text': prompts, 'label': train_data[CLASS]})
})

# train, validation and test split
train_test_dataset = dataset_dict['train'].train_test_split(test_size=0.2, seed=42)

dataset_dict['train'] = train_test_dataset['train']
dataset_dict['validation'] = train_test_dataset['test']

In [141]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 579
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 145
    })
})

In [ ]:
sentence = """

**Social**

"""

items = ['Deliberation', 'Social']

# find the item that appears first inside sentence 
def find_first(sentence, items):
    first = len(sentence)
    item = None
    for i in items:
        index = sentence.find(i)
        if index != -1 and index < first:
            first = index
            item = i
    return item

find_first(sentence, items)

In [2]:
dataset = load_dataset("imdb", split="train")

In [3]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
dataset = load_from_disk("./preprocessed/dataset_Discussion_with_history")